100.00MB를 초과하면 github에 못 올린다는데 220326-1 파일 이후 올릴수가 없어서 폴더를 새로 만들었다

## 사라지는 경사
순환신경망 문제점 - 사라지는 경사(딥러닝 전반에 걸쳐서 나타나는 문제)  
네트워크가 깊어지면 마지막 출력에서 오차가 발생하면 거슬러 올라오는 경우

## LSTM과 GRU
순환신경망에서 출력은 활성화 함수를 한번 거친다. 그러나 LSTM이랑 GRU는 거치지 않고 바로 넘겨준다  
LSTM - 옆으로 넘어가는 신호가 2개 있다. 하나는 처리를 거치고 하나는 거치지 않는다  
GRU - LSTM 단순화.  

## 순환신경망의 방향 구조
- 정방향, 역방향, 양방향(두 개의 신경망으로 각각 처리한 후 합쳐서 내보냄)

## 순환신경망의 문제
앞에 처리가 끝나야 다음 처리를 할 수 있다. - 순서대로 계산하므로 병렬 계산 불가. 계산 속도 느림  
긴 문맥의 학습은 잘 되지 않음


# 감성분석 실습

# 1. 데이터 준비

In [3]:
# 텐서플로우, 순환신경망, 감성분석
import pandas as pd
df = pd.read_csv(r'C:\Users\Owner\Desktop\khh\git\ICT\imdb.zip')

In [4]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
import joblib
tk = joblib.load(r'C:\Users\Owner\Desktop\khh\git\ICT\tokenizer.pkl')

In [6]:
from sklearn.model_selection import train_test_split
review_train, review_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],
                                                             test_size=0.2, random_state=42)

In [7]:
seqs = tk.texts_to_sequences(review_train)

In [8]:
review_train.iloc[0]

'It is an insane game.'

In [9]:
seqs[0]    # 위의 문장을 숫자로 바꾸면

[9, 6, 33, 1258, 214]

In [10]:
seqs[:3]

[[9, 6, 33, 1258, 214],
 [178, 5, 28, 35, 23, 168, 713, 591, 3, 713, 1, 10, 1, 280],
 [206, 336, 4]]

# 2. 순방향 순환신경망

In [15]:
import tensorflow as tf
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=None,
                                                     padding='pre', truncating='pre')
# maxlen 기본값은 0, 만약 10을 넣어주면 10개까지만 받고 잘라줌
# padding = pre : 앞에 0으로 채워줌
# truncating: maxlen이 지정되어 있는 경우만 적용, pre니까 maxlen길이 넘어가면 앞에서 자름

In [16]:
pads[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    9,    6,   33, 1258,  214],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  178,    5,   28,   35,   23,  168,  713,
         591,    3,  713,    1,   10,    1,  280],
       [  

In [17]:
# 단어의 개수
NUM_WORDS = tk.num_words + 1
NUM_WORDS

2001

In [21]:
# 모델 만들기
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),# Output Shape에 보면(800, 73, 8)이다. (800, 73)의 문장을 받은거니까
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
len(pads), pads.shape

(800, (800, 73))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 8)           16008     
_________________________________________________________________
lstm_1 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 11ms/step - loss: 0.6929 - accuracy: 0.5362
Epoch 2/10
25/25 [==============================] - 0s 11ms/step - loss: 0.6895 - accuracy: 0.7212
Epoch 3/10
25/25 [==============================] - 0s 11ms/step - loss: 0.6817 - accuracy: 0.7825
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6595 - accuracy: 0.8000
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.5821 - accuracy: 0.8175
Epoch 6/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4723 - accuracy: 0.8375
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3598 - accuracy: 0.9162
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2838 - accuracy: 0.9400
Epoch 9/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2302 - accuracy: 0.9575
Epoch 10/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2078 - accuracy: 0.9625


# 3. 역방향 순환신경망

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),    # go_backwords=True : 역방향 순환신경망
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [30]:
# 나머지는 순방향 순환신경망과 동일하다
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 10ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6897 - accuracy: 0.6275
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6832 - accuracy: 0.7200
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6672 - accuracy: 0.7663
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6138 - accuracy: 0.8475
Epoch 6/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4894 - accuracy: 0.8587
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3849 - accuracy: 0.8950
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3130 - accuracy: 0.9275
Epoch 9/10
25/25 [==============================] - 0s 11ms/step - loss: 0.2602 - accuracy: 0.9413
Epoch 10/10
25/25 [==============================] - 0s 11ms/step - loss: 0.2207 - accuracy: 0.9575


In [32]:
# padding='post'로
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post')

In [33]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 10ms/step - loss: 0.6926 - accuracy: 0.5487
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6854 - accuracy: 0.7400
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6630 - accuracy: 0.7600
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.5924 - accuracy: 0.8438
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4764 - accuracy: 0.8863
Epoch 6/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3712 - accuracy: 0.9262
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2918 - accuracy: 0.9413
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2673 - accuracy: 0.9400
Epoch 9/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2309 - accuracy: 0.9563
Epoch 10/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2021 - accuracy: 0.9625


# 4. 양방향 순환신경망

In [36]:
# Bidirectional(tf.keras.layers.LSTM(8)) : 양방향 순환신경망
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [37]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 8)           16008     
_________________________________________________________________
bidirectional (Bidirectional (None, 16)                1088      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 17,113
Trainable params: 17,113
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 3s 12ms/step - loss: 0.6920 - accuracy: 0.5387
Epoch 2/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6854 - accuracy: 0.7375
Epoch 3/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6698 - accuracy: 0.7912
Epoch 4/10
25/25 [==============================] - 0s 12ms/step - loss: 0.6212 - accuracy: 0.8175
Epoch 5/10
25/25 [==============================] - 0s 12ms/step - loss: 0.4815 - accuracy: 0.8625
Epoch 6/10
25/25 [==============================] - 0s 12ms/step - loss: 0.3264 - accuracy: 0.9062
Epoch 7/10
25/25 [==============================] - 0s 12ms/step - loss: 0.2268 - accuracy: 0.9513
Epoch 8/10
25/25 [==============================] - 0s 12ms/step - loss: 0.1681 - accuracy: 0.9650
Epoch 9/10
25/25 [==============================] - 0s 12ms/step - loss: 0.1355 - accuracy: 0.9700
Epoch 10/10
25/25 [==============================] - 0s 12ms/step - loss: 0.1157 - accuracy: 0.9750


# 자연어 생성 소개
자연어 처리의 두 분야 - 자연어 이해/생성
  
  
## 자연어 이해
일반 사람의 언어에서 듣기나 읽기에 해당  
사람의 언어로 듣거나 텍스트로 입력 받음 ex) 감성 분석 - 텍스트를 긍/부정으로 분류하거나 점수 매김
  
  
## 자연어 생성
일반 사람의 언어에서 말하거나 쓰기에 해당  
자연어 생성 방법 : 규칙기반 / 기계학습 / 혼합  
### 규칙 기반
- 자연어 생성을 여러 단계로 분할
- 장점 : 일일히 프로그램을 짜기 때문에 각각의 과정을 세세하게 구성할 수 있다
- 단점 : 일일히 프로그램을 짜기 때문에 많은 노력이 필요하고 자연스러운 문장 만들기 어렵다.